## Imports

In [215]:
# Data
import numpy as np
import pandas as pd

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

## Data Frames

In [216]:
df_2015 = pd.read_csv('data/2015.csv')
df_2016 = pd.read_csv('data/2016.csv')
df_2017 = pd.read_csv('data/2017.csv')

In [217]:
df_2016.head(3)

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Denmark,Western Europe,1,7.526,7.460,7.592,1.44178,1.16374,0.79504,0.57941,0.44453,0.36171,2.73939
1,Switzerland,Western Europe,2,7.509,7.428,7.590,1.52733,1.14524,0.86303,0.58557,0.41203,0.28083,2.69463
2,Iceland,Western Europe,3,7.501,7.333,7.669,1.42666,1.18326,0.86733,0.56624,0.14975,0.47678,2.83137


In [218]:
df_2017.head(3)

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707
2,Iceland,3,7.504,7.622030,7.385970,1.480633,1.610574,0.833552,0.627163,0.475540,0.153527,2.322715


### Adding `Region` back in to `df_2017`

In [219]:
def region_dict(pd_data_frame):
    
    """ This function captures the region of a country from previous data frames """
    
    output = dict()
    
    for _, row in pd_data_frame.iterrows():
        if row["Country"] not in output:
            output[row["Country"]] = row["Region"]
    
    return output

In [220]:
dict_1 = region_dict(df_2015)
dict_2 = region_dict(df_2016)

In [221]:
def join_dicts(dict_1, dict_2):
    
    """ This fucntion will join 2 dictionaries and remove duplicates """
    
    for key, value in dict_2.items():
        if key not in dict_1:
            dict_1[key] = value
    
    return dict_1

dict_3 = join_dicts(dict_1, dict_2)

In [222]:
def add_region(pd_data_frame):
    
    """ This function will add the region back into the dataframe """
    # set regions series     
    regions = []
    for country in pd_data_frame["Country"]:
        try: 
            regions.append(dict_3[country])
        except:
            regions.append("N/A")
            
    # set region column to regions series    
    pd_data_frame['Region'] = regions
    return pd_data_frame

df_2017 = add_region(df_2017)

df_2017.head(2)

,Country,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual,Region
0,Norway,1,7.537,7.594445,7.479556,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027,Western Europe
1,Denmark,2,7.522,7.581728,7.462272,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707,Western Europe


### I don't like the way `df_2017` is formatted!

In [223]:
df_2017 = df_2017[[
    "Country","Region","Happiness.Rank","Happiness.Score","Whisker.low","Whisker.high","Economy..GDP.per.Capita.",
    "Family","Health..Life.Expectancy.","Freedom","Generosity","Trust..Government.Corruption.","Dystopia.Residual"
                  ]]

In [224]:
df_2017.head(2)

,Country,Region,Happiness.Rank,Happiness.Score,Whisker.low,Whisker.high,Economy..GDP.per.Capita.,Family,Health..Life.Expectancy.,Freedom,Generosity,Trust..Government.Corruption.,Dystopia.Residual
0,Norway,Western Europe,1,7.537,7.479556,7.594445,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,Western Europe,2,7.522,7.462272,7.581728,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707


In [225]:
df_2017.columns = df_2016.columns

In [226]:
df_2017.head(2)

,Country,Region,Happiness Rank,Happiness Score,Lower Confidence Interval,Upper Confidence Interval,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,Norway,Western Europe,1,7.537,7.479556,7.594445,1.616463,1.533524,0.796667,0.635423,0.362012,0.315964,2.277027
1,Denmark,Western Europe,2,7.522,7.462272,7.581728,1.482383,1.551122,0.792566,0.626007,0.355280,0.400770,2.313707


## What can I predict?

- I will predict `Happiness Score` based off of GDP, Family, Health, Freedom, Trust, Generosity & Dystopia Residual :)
- Will most likely be a multi-variable regression model.

In [227]:
# Dropping unnecessary cells
df_2015.drop(['Country', 'Region', 'Happiness Rank', 'Standard Error'], axis=1, inplace=True)
df_2016.drop(['Country', 'Region', 'Happiness Rank', 'Lower Confidence Interval', 'Upper Confidence Interval'], axis=1, inplace=True)
df_2017.drop(['Country', 'Region', 'Happiness Rank', 'Lower Confidence Interval', 'Upper Confidence Interval'], axis=1, inplace=True)

In [232]:
data_frames = [df_2015, df_2016, df_2017]
df_merged = pd.concat(data_frames)
df_merged.head(3)

,Happiness Score,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),Generosity,Dystopia Residual
0,7.587,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738
1,7.561,1.30232,1.40223,0.94784,0.62877,0.14145,0.43630,2.70201
2,7.527,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204


## Data Normalisation

In [228]:
min_max_scaler = preprocessing.MinMaxScaler()
X_minmax = min_max_scaler.fit_transform(housing_features)
print(X_minmax)

NameError: name 'housing_features' is not defined

## Data Scaling

## Splitting Data

## Training Data